# Aufgabe 1

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

In [3]:
matches = pd.read_csv("./data/atp_matches_till_2022.csv")
matches

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1968-2029,Dublin,Grass,32,A,19680708,270,112411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-2029,Dublin,Grass,32,A,19680708,271,126914,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-2029,Dublin,Grass,32,A,19680708,272,209523,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-2029,Dublin,Grass,32,A,19680708,273,100084,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-2029,Dublin,Grass,32,A,19680708,274,100132,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188156,2022-M-DC-2022-WG2-PO-GRE-JAM-01,Davis Cup WG2 PO: GRE vs JAM,Clay,4,D,20220304,4,209362,NaN,NaN,...,68.0,42.0,12.0,10.0,11.0,13.0,1103.0,9.0,1130.0,8.0
188157,2022-M-DC-2022-WG2-PO-GRE-JAM-01,Davis Cup WG2 PO: GRE vs JAM,Clay,4,D,20220304,5,202065,NaN,NaN,...,56.0,40.0,20.0,15.0,4.0,8.0,808.0,23.0,1390.0,4.0
188158,2022-M-DC-2022-WG2-PO-HKG-BEN-01,Davis Cup WG2 PO: HKG vs BEN,Hard,4,D,20220304,1,138846,NaN,NaN,...,54.0,29.0,8.0,11.0,6.0,10.0,1059.0,10.0,1881.0,1.0
188159,2022-M-DC-2022-WG2-PO-HKG-BEN-01,Davis Cup WG2 PO: HKG vs BEN,Hard,4,D,20220304,2,209409,NaN,NaN,...,39.0,24.0,7.0,10.0,5.0,9.0,NaN,NaN,NaN,NaN


In [4]:
match_lineups: pd.DataFrame = matches.loc[:, ["winner_ioc", "loser_ioc"]]
match_lineups.dropna(
    inplace=True)  # Nicht alle Spiele haben einen Gewinner und Verlierer, da z.B. Spiele abgebrochen werden
match_lineups.count()

winner_ioc    188085
loser_ioc     188085
dtype: int64

In [5]:
win_counts = match_lineups.groupby(["winner_ioc"]).value_counts()
loss_count = match_lineups.groupby(["loser_ioc"]).value_counts()
win_counts = win_counts.reset_index(name="wins")
loss_count = loss_count.reset_index(name="losses")
display(win_counts)
display(loss_count)
display(win_counts.loc[(win_counts["loser_ioc"] == "USA") & (win_counts["winner_ioc"] == "USA")])
display(loss_count.loc[(loss_count["loser_ioc"] == "USA") & (loss_count["winner_ioc"] == "USA")])

,winner_ioc,loser_ioc,wins
0,AHO,JAM,8
1,AHO,GUA,6
2,AHO,ESA,2
3,AHO,HAI,2
4,AHO,DOM,2
...,...,...,...
5410,ZIM,SYR,1
5411,ZIM,GEO,1
5412,ZIM,ECU,1
5413,ZIM,MON,1


,loser_ioc,winner_ioc,losses
0,AHO,PAR,7
1,AHO,ECU,7
2,AHO,GUA,6
3,AHO,MEX,4
4,AHO,BRA,4
...,...,...,...
5410,ZIM,NZL,1
5411,ZIM,PER,1
5412,ZIM,TUR,1
5413,ZIM,UNK,1


,winner_ioc,loser_ioc,wins
5091,USA,USA,13741


,loser_ioc,winner_ioc,losses
5110,USA,USA,13741


In [6]:
win_counts["losses"] = \
win_counts.merge(loss_count, how="left", left_on=["winner_ioc", "loser_ioc"], right_on=["loser_ioc", "winner_ioc"])[
    "losses"]
win_counts["losses"].fillna(0, inplace=True)
win_counts["total"] = win_counts["wins"] + win_counts["losses"]
win_counts["win_rate"] = win_counts["wins"] / win_counts["total"]
win_counts["loss_rate"] = 1 - win_counts["win_rate"]

# Calculate the confidence interval for the win rate. This is done using the Wilson score interval
# https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval#Wilson_score_interval
win_counts["win_rate_ci"] = 1.96 * np.sqrt(
    (win_counts["win_rate"] * (1 - win_counts["win_rate"])) / win_counts["total"])
win_counts["win_rate_lower"] = win_counts["win_rate"] - win_counts["win_rate_ci"]
win_counts["win_rate_upper"] = win_counts["win_rate"] + win_counts["win_rate_ci"]

win_counts["loss_rate_ci"] = 1.96 * np.sqrt(
    (win_counts["loss_rate"] * (1 - win_counts["loss_rate"])) / win_counts["total"])
win_counts["loss_rate_lower"] = win_counts["loss_rate"] - win_counts["loss_rate_ci"]
win_counts["loss_rate_upper"] = win_counts["loss_rate"] + win_counts["loss_rate_ci"]

# Clip the confidence interval to 0 and 1
win_counts["win_rate_lower"] = win_counts["win_rate_lower"].clip(lower=0)
win_counts["win_rate_upper"] = win_counts["win_rate_upper"].clip(upper=1)
win_counts["loss_rate_lower"] = win_counts["loss_rate_lower"].clip(lower=0)
win_counts["loss_rate_upper"] = win_counts["loss_rate_upper"].clip(upper=1)

# Remove extreme values
win_counts = win_counts.loc[(win_counts["win_rate_ci"] != 0)]
win_counts = win_counts.loc[(win_counts["loss_rate_ci"] != 0)]

win_counts.sort_values(by="win_rate", ascending=False, inplace=True)
win_counts

,winner_ioc,loser_ioc,wins,losses,total,win_rate,loss_rate,win_rate_ci,win_rate_lower,win_rate_upper,loss_rate_ci,loss_rate_lower,loss_rate_upper
4110,RSA,IRL,27,1.0,28.0,0.964286,0.035714,0.068739,0.895547,1.000000,0.068739,0.000000,0.104453
5157,USA,CHN,21,1.0,22.0,0.954545,0.045455,0.087043,0.867503,1.000000,0.087043,0.000000,0.132497
200,AUS,IRL,33,2.0,35.0,0.942857,0.057143,0.076900,0.865957,1.000000,0.076900,0.000000,0.134043
2063,GER,UNK,16,1.0,17.0,0.941176,0.058824,0.111852,0.829325,1.000000,0.111852,0.000000,0.170675
1644,ESP,MON,30,2.0,32.0,0.937500,0.062500,0.083870,0.853630,1.000000,0.083870,0.000000,0.146370
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3339,MON,ESP,2,30.0,32.0,0.062500,0.937500,0.083870,0.000000,0.146370,0.083870,0.853630,1.000000
5004,UNK,GER,1,16.0,17.0,0.058824,0.941176,0.111852,0.000000,0.170675,0.111852,0.829325,1.000000
2470,IRL,AUS,2,33.0,35.0,0.057143,0.942857,0.076900,0.000000,0.134043,0.076900,0.865957,1.000000
971,CHN,USA,1,21.0,22.0,0.045455,0.954545,0.087043,0.000000,0.132497,0.087043,0.867503,1.000000


In [7]:
match_lineups.loc[(match_lineups["winner_ioc"] == "UNK") & (match_lineups["loser_ioc"] == "GER")]

,winner_ioc,loser_ioc
4618,UNK,GER


In [8]:
win_counts.loc[(win_counts["winner_ioc"] == "UNK") & (win_counts["loser_ioc"] == "GER")]

,winner_ioc,loser_ioc,wins,losses,total,win_rate,loss_rate,win_rate_ci,win_rate_lower,win_rate_upper,loss_rate_ci,loss_rate_lower,loss_rate_upper
5004,UNK,GER,1,16.0,17.0,0.058824,0.941176,0.111852,0.0,0.170675,0.111852,0.829325,1.0


In [25]:
win_counts.sort_values(by="win_rate", ascending=True, inplace=True)
px.density_heatmap(win_counts, x="winner_ioc", y="loser_ioc", z="win_rate", color_continuous_scale="Viridis",
                   title="Win Rate by Country", height=600, width=600, template="plotly_white")

In [10]:
# Germanys win rate against other countries
germanys_wins = win_counts.loc[win_counts["winner_ioc"] == "GER"].sort_values(by="win_rate", ascending=False)
px.bar(germanys_wins, x="loser_ioc", y="win_rate", title="Deutschlands Gewinn Rate gegenüber anderer Länder",
       height=600, width=600, range_y=[0, 1], labels={
        "loser_ioc": "Land",
        "win_rate": "Gewinn Rate"
    }, error_y="win_rate_ci", template="plotly_white").add_hline(y=0.5, line_dash="dot", line_color="red")

In [11]:
germanys_losses = win_counts.loc[win_counts["loser_ioc"] == "GER"].sort_values(by="win_rate", ascending=False)
px.bar(germanys_losses, x="winner_ioc", y="win_rate", title="Deutschlands Verlust Rate gegenüber anderer Länder",
       height=600, width=600, range_y=[0, 1], labels={
        "winner_ioc": "Land",
        "win_rate": "Verlust Rate"
    }, error_y="loss_rate_ci", template="plotly_white").add_hline(y=0.5, line_dash="dot", line_color="red")

In [12]:
loved_countries = germanys_wins.loc[germanys_wins["win_rate_lower"] > 0.5]
px.bar(loved_countries, x="loser_ioc", y="win_rate", title="Deutschlands Lieblings Gegner", range_y=[0, 1], labels={
    "loser_ioc": "Land",
    "win_rate": "Gewinn Rate"
}, error_y="win_rate_ci", template="plotly_white").add_hline(y=0.5, line_dash="dot", line_color="red")

In [13]:
feared_countries = germanys_wins.loc[germanys_wins["win_rate_upper"] < 0.5]
display(feared_countries)
px.bar(feared_countries, x="loser_ioc", y="win_rate", title="Deutschlands gefürchtetsten Gegner", range_y=[0, 1], labels={
    "loser_ioc": "Land",
    "win_rate": "Gewinn Rate"
}, error_y="loss_rate_ci", template="plotly_white").add_hline(y=0.5, line_dash="dot", line_color="red")

,winner_ioc,loser_ioc,wins,losses,total,win_rate,loss_rate,win_rate_ci,win_rate_lower,win_rate_upper,loss_rate_ci,loss_rate_lower,loss_rate_upper
2012,GER,USA,1256,1464.0,2720.0,0.461765,0.538235,0.018736,0.443029,0.480500,0.018736,0.519500,0.556971
2017,GER,SWE,474,570.0,1044.0,0.454023,0.545977,0.030202,0.423821,0.484225,0.030202,0.515775,0.576179
2018,GER,CZE,373,457.0,830.0,0.449398,0.550602,0.033842,0.415556,0.483239,0.033842,0.516761,0.584444
2021,GER,RUS,307,392.0,699.0,0.439199,0.560801,0.036792,0.402407,0.475991,0.036792,0.524009,0.597593
2025,GER,CRO,189,248.0,437.0,0.432494,0.567506,0.046451,0.386044,0.478945,0.046451,0.521055,0.613956
2019,GER,ARG,358,478.0,836.0,0.428230,0.571770,0.033543,0.394687,0.461773,0.033543,0.538227,0.605313
2015,GER,ESP,691,945.0,1636.0,0.422372,0.577628,0.023935,0.398436,0.446307,0.023935,0.553693,0.601564
2032,GER,SRB,95,133.0,228.0,0.416667,0.583333,0.063994,0.352672,0.480661,0.063994,0.519339,0.647328
2034,GER,CHI,87,122.0,209.0,0.416268,0.583732,0.066831,0.349437,0.483099,0.066831,0.516901,0.650563
2027,GER,SUI,169,245.0,414.0,0.408213,0.591787,0.047346,0.360867,0.455558,0.047346,0.544442,0.639133


In [14]:
loved_countries_names = loved_countries["loser_ioc"].to_list()
feared_countries_names = feared_countries["loser_ioc"].to_list()
display(loved_countries_names)
display(feared_countries_names)

['UNK',
 'VEN',
 'ALG',
 'BOL',
 'TPE',
 'PHI',
 'THA',
 'SLO',
 'DEN',
 'MEX',
 'RSA',
 'ITA']

['USA',
 'SWE',
 'CZE',
 'RUS',
 'CRO',
 'ARG',
 'ESP',
 'SRB',
 'CHI',
 'SUI',
 'MAR',
 'UZB',
 'CYP']

In [15]:
# Are the loved countries worse than germany on average?
loved_countries_win_rate = win_counts.loc[win_counts["winner_ioc"].isin(loved_countries_names)].groupby(
    ["winner_ioc"])
loved_countries_win_rate = loved_countries_win_rate["win_rate"].mean()
loved_countries_win_rate = loved_countries_win_rate.sort_values(ascending=False)

germany_win_rate = win_counts.loc[win_counts["winner_ioc"] == "GER"]["win_rate"].mean()
print(f"Deutschlands Gewinn Rate: {germany_win_rate}")
display(loved_countries_win_rate)

Deutschlands Gewinn Rate: 0.5352316571594513


winner_ioc
MEX    0.531258
RSA    0.530168
THA    0.523257
ITA    0.508490
DEN    0.508268
SLO    0.485427
TPE    0.418088
PHI    0.402079
BOL    0.388554
VEN    0.388395
ALG    0.363370
UNK    0.255693
Name: win_rate, dtype: float64

In [16]:
px.bar(loved_countries_win_rate, x=loved_countries_win_rate.index, y="win_rate", title="Durchschnittliche Gewinn Rate der Lieblings Gegner",
       range_y=[0, 1], labels={
        "index": "Land",
        "win_rate": "Gewinn Rate"
    }, template="plotly_white").add_hline(y=germany_win_rate, line_dash="dot", line_color="red")

In [17]:
# Are the feared countries better than germany on average?
feared_countries_win_rate = win_counts.loc[win_counts["winner_ioc"].isin(feared_countries_names)].groupby(
    ["winner_ioc"])
feared_countries_win_rate = feared_countries_win_rate["win_rate"].mean()
feared_countries_win_rate.sort_values(ascending=False, inplace=True)

germany_win_rate = win_counts.loc[win_counts["winner_ioc"] == "GER"]["win_rate"].mean()
print(f"Deutschlands Gewinn Rate: {germany_win_rate}")
display(feared_countries_win_rate)

Deutschlands Gewinn Rate: 0.5352316571594513


winner_ioc
SUI    0.626522
SRB    0.615470
RUS    0.592856
CRO    0.592566
ESP    0.591328
USA    0.585516
SWE    0.578912
CHI    0.571022
ARG    0.551163
CZE    0.547803
CYP    0.501459
MAR    0.493944
UZB    0.433023
Name: win_rate, dtype: float64

In [18]:
px.bar(feared_countries_win_rate, x=feared_countries_win_rate.index, y="win_rate", title="Durchschnittliche Gewinn Rate der gefürchtetsten Gegner",
       range_y=[0, 1], labels={
        "index": "Land",
        "win_rate": "Gewinn Rate"
    }, template="plotly_white").add_hline(y=germany_win_rate, line_dash="dot", line_color="red")